In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [9]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)


In [10]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [11]:
data


DataFrame[movieId: int, rating: double, userId: int]

In [12]:
data.printSchema()


root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [13]:
from pyspark.sql.functions import *

most_popular = data\
    .groupBy("movieId")\
    .agg(count("userId"))\
    .withColumnRenamed("count(userId)", "num_ratings")\
    .sort(desc("num_ratings"))


In [14]:
most_popular.show(10)


+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|     51|         20|
|     22|         20|
|     50|         20|
|      6|         20|
|     29|         20|
|     94|         19|
|     15|         19|
|     68|         19|
|     55|         19|
|      2|         19|
+-------+-----------+
only showing top 10 rows



In [15]:
most_popular_movies = most_popular.join(
    movies, most_popular.movieId == movies.movieId)
most_popular_movies.show(20, truncate=False)


NameError: name 'movies' is not defined

In [ ]:
top_rated = ratings\
    .groupBy("movieId")\
    .agg(avg(col("rating")))\
    .withColumnRenamed("avg(rating)", "avg_rating")\
    .sort(desc("avg_rating"))


In [ ]:
top_rated = ratings\
    .groupBy("movieId")\
    .agg(count("userId"), avg(col("rating")))\
    .withColumnRenamed("count(userId)", "num_ratings")\
    .withColumnRenamed("avg(rating)", "avg_rating")


In [ ]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId).sort(
    desc("avg_rating"), desc("num_ratings"))
top_rated_movies.show(10)


In [ ]:
top_rated_movies.select([mean('num_ratings'), min(
    'num_ratings'), max('num_ratings')]).show(1)


In [ ]:
top_rated_movies.approxQuantile('num_ratings', [0.5], 0)


In [ ]:
top_rated_movies.approxQuantile('num_ratings', [0.25], 0)


In [ ]:
top_rated_movies.approxQuantile('num_ratings', [0.75], 0)


In [ ]:
top_rated_movies.where("num_ratings > 500").show(20, truncate=False)


In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [ ]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [ ]:
training.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|   1.0|     3|
|      0|   1.0|     5|
|      0|   1.0|     6|
|      0|   1.0|    11|
|      0|   1.0|    13|
|      0|   1.0|    19|
|      0|   1.0|    20|
|      0|   1.0|    21|
|      0|   1.0|    22|
|      0|   1.0|    23|
|      0|   3.0|    10|
|      0|   3.0|    28|
|      1|   1.0|     3|
|      1|   1.0|     4|
|      1|   1.0|     5|
|      1|   1.0|     6|
|      1|   1.0|     7|
|      1|   1.0|    14|
|      1|   1.0|    18|
|      1|   1.0|    19|
+-------+------+------+
only showing top 20 rows



In [ ]:
test.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|   1.0|     8|
|      0|   1.0|    15|
|      0|   1.0|    26|
|      0|   1.0|    27|
|      1|   1.0|    26|
|      1|   1.0|    28|
|      1|   4.0|    15|
|      2|   1.0|    15|
|      2|   1.0|    23|
|      2|   2.0|     7|
|      2|   3.0|     0|
|      2|   3.0|     6|
|      3|   1.0|     0|
|      3|   1.0|     9|
|      3|   1.0|    13|
|      3|   1.0|    26|
|      3|   1.0|    28|
|      3|   2.0|     8|
|      4|   1.0|     9|
|      4|   1.0|    23|
+-------+------+------+
only showing top 20 rows



In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|-0.86268294|
|      3|   1.0|    28|  2.3682623|
|      0|   1.0|    26|   0.736683|
|      0|   1.0|    27|  1.6588295|
|      1|   1.0|    26|  2.1351407|
|      3|   1.0|    26| -0.7142657|
|      3|   1.0|    13| -0.2651818|
|      4|   2.0|    13|  2.6796756|
|      2|   3.0|     6|   -1.17831|
|      0|   1.0|    15|  1.2747415|
|      1|   4.0|    15|  1.2740507|
|      2|   1.0|    15|  2.3288383|
|      3|   1.0|     9|  0.0854869|
|      4|   1.0|     9| 0.60577434|
|      0|   1.0|     8|  1.0385127|
|      3|   2.0|     8|  2.9446602|
|      2|   1.0|    23|  0.5978978|
|      4|   1.0|    23|0.065588444|
|      2|   2.0|     7| 0.99254626|
|      2|   3.0|     0|  0.6517123|
+-------+------+------+-----------+
only showing top 20 rows



In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8091418734212048


In [ ]:
single_user = test.filter(test['userId']==26).select(['movieId','userId'])

In [ ]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    26|
|      1|    26|
|      3|    26|
|      7|    26|
|     21|    26|
|     22|    26|
|     24|    26|
|     27|    26|
|     40|    26|
|     45|    26|
|     47|    26|
|     49|    26|
|     50|    26|
|     54|    26|
|     61|    26|
|     68|    26|
|     73|    26|
|     76|    26|
|     95|    26|
+-------+------+



In [ ]:
reccomendations = model.transform(single_user)

In [ ]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     54|    26| 6.5735664|
|     47|    26| 5.2101517|
|      7|    26| 3.0717003|
|     50|    26| 2.8704505|
|     76|    26|  2.576686|
|      1|    26| 2.1351407|
|     95|    26| 1.7132492|
|     73|    26| 1.1988099|
|     49|    26|0.98570096|
|     27|    26| 0.9017774|
|      0|    26|  0.736683|
|     21|    26|  0.606101|
|     68|    26|0.41728032|
|     61|    26|0.24670398|
|     22|    26|0.22418928|
|     45|    26|0.07607293|
|     40|    26|-0.3926839|
|      3|    26|-0.7142657|
|     24|    26|-1.1734585|
+-------+------+----------+

